# Item 02. 변수의 스코프를 최소화해라

## (1) 스코프는 좁게

- 프로퍼티보다는 지역변수
- 반복문 내부에서만 변수가 사용된다면, 반복문 내부에 선언
- 이유는 유지보수때문!

In [0]:
// BAD!
val users : Array<String> = arrayOf("1번 유저", "2번유저", "3번유저")
var user : String = ""
for(i in users.indices){
    user = users[i]
    println("user[$i]는 $user")
}

// GOOD
val users2 : Array<String> = arrayOf("1번 유저", "2번유저", "3번유저")
for((i, user2) in users2.withIndex()){
    println("user[$i]는 $user2")
}

## (2) 식(expression)과 조건분해할당(destruction declaration)을 적극적으로사용하자

- 코틀린의강점은 조건문(if else, when)이나 try catch문등이 식으로도 표현된다는점!
- 또한 조건분해할당이 가능하므로 변수선언을 좀더 간결하게 하도록 하자!

In [ ]:

// 이렇게 하면 가독성도 좋고 코드 라인도 짧음!
fun updateWeather(degree : Int) {
    val (description, color) = when {
        degree < 5 -> "cold" to "blue"
        degree > 25 -> "hot" to "red"
        else -> "normal" to "yellow"
    }
}

- 에라토테네스의 체를 시퀸스로 구현하며 잘못된 변수 스코프 지정 확인해보기

In [9]:
// [1] 최초 구현

var numbers = (2..100).toList()
val primes = mutableListOf<Int>()

while (numbers.isNotEmpty()) {
    val first = numbers.first()
    primes.add(first)
    numbers = numbers.filter { it % first != 0 }
}

println(primes)


// [2] 시퀸스로 확장해보기
val sPrimes = sequence {
    var numbers = generateSequence(2) { it + 1 } // 2부터 시작하는 무한 숫자 시퀸스 할당

    while (true) {
        val prime = numbers.first() // 무한 시퀸스에서 첫번째 숫자
        yield(prime) // 해당 숫자 선택(시퀸스 결과값에 포함)
        numbers = numbers.drop(1) // 해당 숫자 제거하고
            .filter { it % prime != 0 } // 제거된 무한숫자 numbers에서 해당 숫자로 나뉠때 나누어 떨어지지 않는 숫자만 추출해서 무한숫자 시퀸스에 재할당
    }
}
println(sPrimes.take(10).toList()) // yield 로 산출되는 결과값이 10개가 나올때까지 시퀸스 실행

// [3] 잘못된 시퀸스로 할당
val sPrimes2 = sequence {
    var numbers = generateSequence(2) { it + 1 } // 2부터 시작하는 무한 숫자 시퀸스 할당

    var prime : Int // val prime 계속할당하는게 낭비같은데? 한번할당하고 재사용해보자! <----
    while (true) {
        prime = numbers.first() // 무한 시퀸스에서 첫번째 숫자
        yield(prime) // 해당 숫자 선택(시퀸스 결과값에 포함)
        numbers = numbers.drop(1) // 해당 숫자 제거하고
            .filter { it % prime != 0 } // 제거된 무한숫자 numbers에서 해당 숫자로 나뉠때 나누어 떨어지지 않는 숫자만 추출해서 무한숫자 시퀸스에 재할당
    }
}
println(sPrimes2.take(10).toList()) // yield 로 산출되는 결과값이 10개가 나올때까지 시퀸스 실행

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
[2, 3, 5, 6, 7, 8, 9, 10, 11, 12]


시퀸스는 자바 stream과 비슷하게 지연 연산을 하므로, 값을 실제로 호출할때 연산하게된다.

![sequence](https://velog.velcdn.com/images/hope0206/post/0e808a98-5092-4cfe-9aa9-53a3f702ffe3/image.png)
